### Load DIC data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.lines as mlines
from defdap.quat import Quat
import defdap.ebsd as ebsd
import defdap.hrdic as hrdic
from IPython.display import display
from matplotlib_scalebar.scalebar import ScaleBar
from skimage.filters import threshold_local, sobel
import skimage.morphology as mph
from skimage import feature, measure
import scipy.ndimage as nd
from skimage.transform import rescale, resize, downscale_local_mean
from scipy import ndimage
import matplotlib as mpl

%matplotlib osx

#Load in DIC maps

DicFilePath = "/Users/mbcx9rt5/Dropbox (Research Group)/python/DefDAP/errordata/"
fieldfree = hrdic.Map(DicFilePath, "fieldfree.TXT")
fieldfree.setCrop(xMin=2,xMax=2,yMin=2,yMax=10)

immersion = hrdic.Map(DicFilePath, "immersion.TXT")
immersion.setCrop(xMin=2,xMax=2,yMin=2,yMax=10)

stagebias = hrdic.Map(DicFilePath, "stagebias.TXT")
stagebias.setCrop(xMin=2,xMax=2,yMin=2,yMax=10)


### Select DIC map

In [2]:
fieldfree.plotMaxShear(vmax=0.01)

In [3]:
print(np.mean(fieldfree.crop(fieldfree.max_shear)*100))
print(np.std(fieldfree.crop(fieldfree.max_shear)*100))

0.313165766098
0.165421641184


In [95]:
f, ((ax1, ax2), (ax3,ax4),(ax5,ax6)) = plt.subplots(3, 2, figsize=(6, 4))

ax1.imshow(fieldfree.crop(fieldfree.f11-1) * 100,cmap='viridis', interpolation='None', vmin=0, vmax=1)
ax2.imshow(fieldfree.crop(fieldfree.f22-1) * 100,cmap='viridis', interpolation='None', vmin=0, vmax=1)
ax3.imshow(stagebias.crop(stagebias.f11-1) * 100,cmap='viridis', interpolation='None', vmin=0, vmax=1)
ax4.imshow(stagebias.crop(stagebias.f22-1) * 100,cmap='viridis', interpolation='None', vmin=0, vmax=1)
ax5.imshow(immersion.crop(immersion.f11-1) * 100,cmap='viridis', interpolation='None', vmin=0, vmax=1)
ax6.imshow(immersion.crop(immersion.f22-1) * 100,cmap='viridis', interpolation='None', vmin=0, vmax=1)

ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)
ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)
ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)
ax4.get_xaxis().set_visible(False)
ax4.get_yaxis().set_visible(False)
ax5.get_xaxis().set_visible(False)
ax5.get_yaxis().set_visible(False)
ax6.get_xaxis().set_visible(False)
ax6.get_yaxis().set_visible(False)

In [122]:
f, ((ax1, ax2), (ax3,ax4)) = plt.subplots(2, 2, figsize=(6, 4))

sel=immersion

ax1.imshow(sel.crop(sel.f11-1) * 100,cmap='coolwarm', interpolation='None', vmin=-0.5, vmax=0.5)
ax2.imshow(sel.crop(sel.f12) * 100,cmap='coolwarm', interpolation='None', vmin=-0.5, vmax=0.5)
ax3.imshow(sel.crop(sel.f21) * 100,cmap='coolwarm', interpolation='None', vmin=-0.5, vmax=0.5)
ax4.imshow(sel.crop(sel.f22-1) * 100,cmap='coolwarm', interpolation='None', vmin=-0.5, vmax=0.5)

ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)
ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)
ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)
ax4.get_xaxis().set_visible(False)
ax4.get_yaxis().set_visible(False)

In [124]:
f, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(6, 4))

sel=immersion

ax1.imshow(sel.crop(sel.x_map),cmap='coolwarm', interpolation='None', vmin=-2, vmax=2)
ax2.imshow(sel.crop(sel.y_map),cmap='rainbow', interpolation='None', vmin=-2, vmax=2)

ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)
ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)

In [55]:
data1 = [item for sublist in fieldfree.crop(fieldfree.max_shear)*100 for item in sublist]
data2 = [item for sublist in immersion.crop(immersion.max_shear)*100 for item in sublist]
data3 = [item for sublist in stagebias.crop(stagebias.max_shear)*100 for item in sublist]

f, (ax1) = plt.subplots(1, figsize=(6, 4))

counts, bins, bars = ax1.hist(data1, bins=np.linspace(0,1.5,100), color='r', histtype='step',  label='Field-free')
plt.xlabel("Effective shear strain (%)", fontsize="12")
plt.ylabel("Frequency", fontsize="12")
center = (bins[:-1] + bins[1:]) / 2

counts3, bins, bars = ax1.hist(data3, bins=np.linspace(0,1.5,100), color='b', histtype='step',  label='Field-free + 2kV Bias')
center = (bins[:-1] + bins[1:]) / 2

counts2, bins, bars = ax1.hist(data2, bins=np.linspace(0,1.5,100), color='g', histtype='step',  label='Immersion + 2kV Bias')
center = (bins[:-1] + bins[1:]) / 2

plt.tick_params(top='on', right='on', which='both', direction='in')
plt.tick_params(axis='y', which='minor')
plt.legend()
f.tight_layout()
plt.show()

f.savefig('histograms.pdf',dpi=400, pad_inches=0)

In [98]:
print('MSS')
print(np.mean(fieldfree.crop(fieldfree.max_shear))*100)
print(np.mean(stagebias.crop(stagebias.max_shear))*100)
print(np.mean(immersion.crop(immersion.max_shear))*100)
print(np.std(fieldfree.crop(fieldfree.max_shear))*100)
print(np.std(stagebias.crop(stagebias.max_shear))*100)
print(np.std(immersion.crop(immersion.max_shear))*100)

print('Exx')
print(np.mean(fieldfree.crop(fieldfree.f11-1))*100)
print(np.mean(stagebias.crop(stagebias.f11-1))*100)
print(np.mean(immersion.crop(immersion.f11-1))*100)
print(np.mean(np.absolute(fieldfree.crop(fieldfree.f11-1)))*100)
print(np.mean(np.absolute(stagebias.crop(stagebias.f11-1)))*100)
print(np.mean(np.absolute(immersion.crop(immersion.f11-1)))*100)

print('Eyy')
print(np.mean(fieldfree.crop(fieldfree.f22-1))*100)
print(np.mean(stagebias.crop(stagebias.f22-1))*100)
print(np.mean(immersion.crop(immersion.f22-1))*100)
print(np.mean(np.absolute(fieldfree.crop(fieldfree.f22-1)))*100)
print(np.mean(np.absolute(stagebias.crop(stagebias.f22-1)))*100)
print(np.mean(np.absolute(immersion.crop(immersion.f22-1)))*100)

print('Exy')
print(np.mean(np.absolute(fieldfree.crop(fieldfree.f12)))*100)
print(np.mean(np.absolute(stagebias.crop(stagebias.f12)))*100)
print(np.mean(np.absolute(immersion.crop(immersion.f12)))*100)
print('Eyx')
print(np.mean(np.absolute(fieldfree.crop(fieldfree.f21)))*100)
print(np.mean(np.absolute(stagebias.crop(stagebias.f21)))*100)
print(np.mean(np.absolute(immersion.crop(immersion.f21)))*100)

print('MSS max')
print(np.max(fieldfree.crop(fieldfree.max_shear))*100)
print(np.max(stagebias.crop(stagebias.max_shear))*100)
print(np.max(immersion.crop(immersion.max_shear))*100)

MSS
0.313130318204
0.322325805851
0.177662247595
0.16555382628
0.168971711903
0.0933645814477
Exx
0.000802896433147
0.00200884989731
-0.00208267815421
0.263072351112
0.252295881729
0.142417502567
Eyy
0.0613448308289
0.00766416571898
-0.0638200329505
0.284044991546
0.311097928944
0.190795684909
Exy
0.341172916283
0.33076699248
0.18014003079
Eyx
0.282863638498
0.297794801047
0.191895441279
MSS max
1.25492661545
1.24527514373
0.694459996916
